<h1> Main.ipynb </h1>
<h2> 1. Import libraries and modules </h2>

In [75]:
import numpy as np
import pandas as pd

from src.Data_Retriever import DataRetriever as DatRet

from src.Optimization.Markowitz_PT import MarkowitzPT as MPT

from src.Optimization.Environment import PortfolioEnvironment as PorEnv
from src.Optimization.RLModelCompilation import RL_Model as RLM
# from src.Optimization.NeuralNet import CustomCNNExtractor 

from src.Result.Menchero_OGA import MencheroOGA as MOGA
from src.Result.IndPortResults import GenerateResult as GR
from src.Result.OverviewResults import ResultConveyor as RC

import time

import warnings
warnings.filterwarnings('ignore')

<h2> 2. Define operating variables </h2>

In [3]:
trading_n = 400
history_usage = 504
n_sectors = 6
n_stocks_per_sector = 4

# For RL algorithm
history_usage_RL = 30
rolling_reward_window = 15

<h2> 3. Define stock dataframe </h2>

In [86]:
ticker_df =  pd.DataFrame()
ticker_df["Petroleum"] = ["EQNR.OL", "AKRBP.OL", "SUBC.OL", "BWO.OL",]
ticker_df["Seafood (food)"] = ["ORK.OL", "MOWI.OL", "SALM.OL", "LSG.OL"]
ticker_df["Materials"] = ["NHY.OL", "YAR.OL", "RECSI.OL", "BRG.OL"]  #del this
ticker_df["Technologies"] = ["TEL.OL", "NOD.OL", "ATEA.OL", "BOUV.OL"]
ticker_df["Financial"] = ["STB.OL", "DNB.OL", "GJF.OL", "AKER.OL"]
ticker_df["Shipping"] = ["WAWI.OL", "SNI.OL", "BELCO.OL", "ODF.OL"]
ticker_df

,Petroleum,Seafood (food),Materials,Technologies,Financial,Shipping
0,EQNR.OL,ORK.OL,NHY.OL,TEL.OL,STB.OL,WAWI.OL
1,AKRBP.OL,MOWI.OL,YAR.OL,NOD.OL,DNB.OL,SNI.OL
2,SUBC.OL,SALM.OL,RECSI.OL,ATEA.OL,GJF.OL,BELCO.OL
3,BWO.OL,LSG.OL,BRG.OL,BOUV.OL,AKER.OL,ODF.OL


<h2> 4. Define ESG array </h2>

In [5]:
esg_scores = np.array([36.6, 35.3, 17.9, 18, 
                18, 21.2, 18.7, 29.2, 
                15.7, 25.6, 25.6, 18.4, # Del this
                19.8, 13.8, 18.1, 19, 
                17.2, 14, 17.2, 19.5, 
                19.7, 21.2, 26.8, 19.3])

<h2> 5.-7. (Non-necessary for reproducability) </h2>
<h3>Data retrieval</h3>
<h3>Benchmark optimization (MPT)</h3>
<h3>RL optimization x4</h3>
<br>
All of the data has been generated an stored in csvs. Therefore, I can comment out the code lines below.

In [6]:
# Retrieve data from yf API: y-m-d
# data = DatRet(ticker_df, "2013-01-01", "2024-12-31", history_usage_RL=history_usage_RL)
# # In function below, set log=True to check for data availability
# data.retrieve_data()

In [7]:
# Generate benchmark weights thorugh MPT using Sharpe ratio
# # benchmark = MPT(history_usage, trading_n)
# IMPORTANT: In order to see  the effect of the weights, algo exclude last observation from optimization
# # benchmark.frequency_optimizing()

In [8]:
# objectives = ["Return", "Sharpe", "Sortino", "Sterling", "Return", "Sharpe", "Sortino", "Sterling"]
# esg_compliancy = [True, True, True, True, False, False, False, False]
# # objectives = ["Sharpe"]
# # esg_compliancy = [True]

# for i in range(len(objectives)):
#     reinforcement = RLM(esg_scores, 
#                         objective=objectives[i],
#                         history_usage=history_usage_RL,
#                         rolling_reward_window=rolling_reward_window,
#                         total_timesteps=200,
#                         esg_compliancy=esg_compliancy[i],
#                         )
#     reinforcement.train_model()
#     reinforcement.test_model()

<h2> 8. Attribution analysis </h2>

In [9]:
# paths = ["Return_esg_True", "Sharpe_esg_True",
#          "Sortino_esg_True","Sterling_esg_True",
#          "Return_esg_False", "Sharpe_esg_False",
#          "Sortino_esg_False","Sterling_esg_False",]

# analysis_list = []
# for i in range(len(paths)):
#     att_anal = GR(paths[i],
#             n_sectors, n_stocks_per_sector,
#             trading_n,
#             esg_scores, 
#             ticker_df.columns)
#     att_anal.friple_frequency_analysis()
#     analysis_list.append(att_anal)

<h2> 9. Overview  Result Analysis </h2>

In [10]:
# theta = RC(analysis_list, trading_n)
# theta.convey_results()

<h1> 10. Ad-hoc </h1>

In [11]:
# ret = pd.read_csv("Data/Input/StockReturns.csv")
# weights = pd.read_csv("Data/RL_weights_Sharpe_esg_false.csv")

# train_set = ret.iloc[:int(0.85*len(ret))]
# test_set = ret.iloc[int(0.85*len(ret)):]

In [12]:
import pandas as pd
import numpy as np
import yfinance as yf

In [13]:
start_date = "2006-07-01"
end_date = "2024-03-31"

In [88]:
len(np.array(ticker_df).flatten())

24

In [ ]:
class DataRetrieverNew:

    def __init__(self, start_date, end_date, ticker_df):
        """
        doc string
        """
        self.start_date = start_date
        self.end_date = end_date
        self.ticker_list = np.array(ticker_df).flatten()

        self.master_daterange = pd.Series(np.repeat(pd.date_range(start=self.start_date, end=self.end_date, freq="B"), 2), name="Date")
        self.return_df = pd.DataFrame(index=self.master_daterange)
        self.volume_df = pd.DataFrame(index=self.master_daterange)
        self.rolling_return_df = pd.DataFrame(index=self.master_daterange)
        self.rolling_volatility_df = pd.DataFrame(index=self.master_daterange)


    def z_score(self, arr: np.array):
        """
        doc string
        """
        z_score_arr = (arr - np.mean(arr)) / np.std(arr)

        return z_score_arr



    def retrieve_data(self):
        """
        doc string
        """
        for i in range(0, len(self.ticker_list), 1):
            stock_data = yf.download(self.ticker_list[i], start=start_date, end=end_date)

            date_df = pd.DataFrame(index=self.master_daterange)
            open = stock_data["Open"]
            individual_df = pd.merge(date_df, open, on="Date", how="left")
            inter_df = pd.merge(date_df, stock_data["Close"], on="Date", how="left")
            individual_df[self.ticker_list[i]].iloc[1::2] = inter_df[self.ticker_list[i]].iloc[::2]
            return_series = individual_df.interpolate().ffill().bfill()
            vol_series = pd.merge(date_df, stock_data["Volume"], on="Date", how="left")

            self.return_df[self.ticker_list[i]] = return_series[self.ticker_list[i]]
            self.volume_df[self.ticker_list[i]] = self.z_score(vol_series[self.ticker_list[i]])
            self.rolling_return_df[self.ticker_list[i]] = return_series[self.ticker_list[i]].rolling(40).mean().bfill()
            self.return_df[self.ticker_list[i]] = return_series[self.ticker_list[i]].rolling(40).std().bfill()
        
        master_df = pd.concat([self.return_df, self.volume_df, self.rolling_return_df, self.rolling_volatility_df], axis = 1)
        master_df.to_csv("Mix.csv", index=False)

        print("--- Data retrieved successfully ---")

In [99]:
hotel = DataRetrieverNew(start_date,end_date, ["EQNR.OL", "SNI.OL"] )
hotel.retrieve_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

--- Data retrieved successfully ---


In [18]:
master_daterange = pd.Series(np.repeat(pd.date_range(start=start_date, end=end_date, freq="B"), 2), name="Date")
return_df = pd.DataFrame(index=master_daterange)
volume_df = pd.DataFrame(index=master_daterange)
rolling_return = pd.DataFrame(index=master_daterange)
rolling_volatility_df = pd.DataFrame(index=master_daterange)
apprentice_df = pd.DataFrame(index=master_daterange)
stock_data = yf.download("EQNR.OL", start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [28]:
# return_df

In [ ]:
for i in range(0, len(ticker_df),1):
    date_df = pd.DataFrame(index=master_daterange)
    open = stock_data["Open"]
    individual_df = pd.merge(date_df, open, on="Date", how="left")
    inter_df = pd.merge(date_df, stock_data["Close"], on="Date", how="left")
    individual_df["EQNR.OL"].iloc[1::2] = inter_df["EQNR.OL"].iloc[::2]
    return_series = individual_df.interpolate().ffill().bfill()
    vol_series = pd.merge(date_df, stock_data["Volume"], on="Date", how="left") / 2




In [82]:
vol_df = pd.merge(date_df, stock_data["Volume"], on="Date", how="left") / 2

In [83]:
vol_df

,EQNR.OL
Date,
2006-07-03,1952192.0
2006-07-03,1952192.0
2006-07-04,2014003.0
2006-07-04,2014003.0
2006-07-05,2030154.0
...,...
2024-03-27,617997.0
2024-03-28,NaN
2024-03-28,NaN


In [59]:
stock_data

Price,Close,High,Low,Open,Volume
Ticker,EQNR.OL,EQNR.OL,EQNR.OL,EQNR.OL,EQNR.OL
Date,,,,,
2006-07-03,89.831535,90.331990,88.580400,88.830627,3904384
2006-07-04,88.830650,90.582241,88.580423,90.582241,4028006
2006-07-05,88.079948,89.331084,88.079948,88.330176,4060308
2006-07-06,90.582245,90.582245,89.080882,89.331109,7291150
2006-07-07,93.584938,93.584938,89.831531,90.331986,9803194
...,...,...,...,...,...
2024-03-21,277.540710,280.515046,276.467989,277.930783,2681880
2024-03-22,279.686127,280.758848,275.980395,276.419245,1829393


In [19]:
apprentice_df = pd.DataFrame(index=master_daterange)

#Open data
open = stock_data["Open"]   
apprentice_df = pd.merge(apprentice_df, open, on="Date")

# Close data
close = stock_data["Close"]
close_merge = pd.merge(master_daterange, close, on="Date")

# Every other manual merging
for i in range(1,len(apprentice_df),2):
    apprentice_df.iloc[i,0] = close_merge.iloc[i,1]

return_df = apprentice_df.iloc[:,0]


In [21]:
apprentice_df.columns

Index(['EQNR.OL'], dtype='object')

In [25]:
class DataRetrieverNew:
    def __init__(self, start_date, end_date, ticker_df):
        self.start_date = start_date
        self.end_date = end_date
        self.ticker_list = np.array(ticker_df).flatten()

        # Unique business days (no duplicates)
        self.master_daterange = pd.Series(
            pd.date_range(start=start_date, end=end_date, freq="B"), 
            name="Date"
        )
        
        # Initialize DataFrames
        self.return_df = pd.DataFrame(index=self.master_daterange)
        self.volume_df = pd.DataFrame(index=self.master_daterange)
        self.rolling_return_df = pd.DataFrame(index=self.master_daterange)
        self.rolling_volatility_df = pd.DataFrame(index=self.master_daterange)

    def retrieve_data(self):
        for ticker in self.ticker_list:
            stock_data = yf.download(ticker, start=self.start_date, end=self.end_date)
            
            if stock_data.empty:
                continue  # Skip if no data
            
            # Merge Open and Close into master_daterange
            merged_data = self.master_daterange.to_frame().merge(
                stock_data[["Open", "Close", "Volume"]].reset_index(),
                on="Date",
                how="left"
            )
            
            # Forward-fill missing values (weekends/holidays)
            merged_data[["Open", "Close", "Volume"]] = merged_data[["Open", "Close", "Volume"]].ffill().bfill()
            
            # Calculate returns (daily % change)
            returns = merged_data["Close"].pct_change().fillna(0)
            
            # Store data
            self.return_df[ticker] = returns
            self.volume_df[ticker] = merged_data["Volume"]
            self.rolling_return_df[ticker] = returns.rolling(15).mean().bfill()
            self.rolling_volatility_df[ticker] = returns.rolling(15).std().bfill()
        
        # Combine all DataFrames
        master_df = pd.concat([
            self.return_df, 
            self.volume_df, 
            self.rolling_return_df, 
            self.rolling_volatility_df
        ], axis=1)
        
        master_df.to_csv("Mix.csv", index=True)  # Keep index (dates) in CSV

In [27]:
telia = DataRetrieverNew(start_date, end_date, ticker_df)
telia.retrieve_data()

[*********************100%***********************]  1 of 1 completed


MergeError: Not allowed to merge between different levels. (1 levels on the left, 2 on the right)

In [240]:
apprentice_df.iloc[:,0]

Date
2006-07-03     88.378922
2006-07-03     89.374741
2006-07-04     90.121589
2006-07-04     88.378906
2006-07-05     87.880997
                 ...    
2024-03-25    286.295990
2024-03-26    285.312120
2024-03-26    281.917877
2024-03-27    281.032448
2024-03-27    281.770325
Name: EQNR.OL, Length: 8920, dtype: float64

In [ ]:
for i in range(1,len(final_df),2):
    final_df.iloc[i,0] =  close_merge.iloc[i,0]

In [ ]:
final_df[::2] = close_merge[1::2]

In [ ]:
final_df

,EQNR.OL
Date,
2006-07-03,88.378922
2006-07-03,2006-07-03 00:00:00
2006-07-04,90.121589
2006-07-04,2006-07-04 00:00:00
2006-07-05,87.880997
...,...
2024-03-25,2024-03-25 00:00:00
2024-03-26,285.31212
2024-03-26,2024-03-26 00:00:00


In [ ]:
close_merge = pd.merge(final_df, close)[1::2]

In [ ]:
pd.merge(final_df,open, on="Date")

,EQNR.OL
Date,
2006-07-03,88.378922
2006-07-03,88.378922
2006-07-04,90.121589
2006-07-04,90.121589
2006-07-05,87.880997
...,...
2024-03-25,282.901747
2024-03-26,285.312120
2024-03-26,285.312120
